# Assignment 3: Build a seq2seq model for machine translation.

### Name: [Daksh Bhuva]

### Task: Change LSTM model to Bidirectional LSTM Model， translate English to target language and evaluate using Bleu score.

### Due Date: Tuesday, April 19th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue. 
2. Complete the code in Section 1.1, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

### Hint: 

To implement ```Bi-LSTM```, you will need the following code to build the encoder. Do NOT use Bi-LSTM for the decoder. But there are other codes you need to modify to make it work.

In [65]:
# from keras.layers import Bidirectional, Concatenate, LSTM

# encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
#                                   dropout=0.5, name='encoder_lstm'))
# _, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# state_h = Concatenate()([forward_h, backward_h])
# state_c = Concatenate()([forward_c, backward_c])

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [66]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [67]:
# e.g., filename = 'Data/deu.txt'
filename = '/content/spa.txt'

# e.g., n_train = 20000
n_train = 20000

In [68]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [69]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[youre here] => [estas aqui]
[youre here] => [estais aqui]
[youre late] => [estas retrasado]
[youre lost] => [estas perdido]
[youre mean] => [eres mala]
[youre mean] => [eres mezquino]
[youre mine] => [tu eres mio]
[youre nice] => [eres simpatico]
[youre nuts] => [estas loco]
[youre nuts] => [estas chiflado]


In [70]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]
target_texts = numpy.array(target_texts)

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [71]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 55


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [72]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 55)
shape of target_token_index: 29


In [73]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [74]:
target_texts[100]

'\tno puede ser\n'

In [75]:
decoder_input_seq[100, :]

array([ 6,  8,  3,  1, 17, 14,  2, 15,  2,  1,  5,  2, 10,  7,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [76]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 55, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [77]:
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the LSTM layer
encoder_lstm = LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm')
_, state_h, state_c = encoder_lstm(encoder_inputs)

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [78]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer)  [(None, None, 28)]       0         
                                                                 
 encoder_lstm (LSTM)         [(None, 256),             291840    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 291,840
Trainable params: 291,840
Non-trainable params: 0
_________________________________________________________________


### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [79]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [80]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 256)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 256)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 256),  293888      ['decoder_input_x[0][0]',        
                                 (None, 256),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [81]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [82]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 256),        291840      ['encoder_input_x[0][0]']        
                                 (None, 256)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 256),  293888      ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [83]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 55, 30)
shape of decoder_target_data(20000, 55, 30)


In [21]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 10s 13ms/step - loss: 0.9186 - val_loss: 0.8585
Epoch 2/50
250/250 [==============================] - 2s 10ms/step - loss: 0.7349 - val_loss: 0.7513
Epoch 3/50
250/250 [==============================] - 2s 10ms/step - loss: 0.7009 - val_loss: 0.7165
Epoch 4/50
250/250 [==============================] - 2s 10ms/step - loss: 0.6788 - val_loss: 0.6856
Epoch 5/50
250/250 [==============================] - 2s 9ms/step - loss: 0.6618 - val_loss: 0.6597
Epoch 6/50
250/250 [==============================] - 2s 10ms/step - loss: 0.6494 - val_loss: 0.6414
Epoch 7/50
250/250 [==============================] - 2s 9ms/step - loss: 0.6383 - val_loss: 0.6260
Epoch 8/50
250/250 [==============================] - 2s 10ms/step - loss: 0.6280 - val_loss: 0.6153
Epoch 9/50
250/250 [==============================] - 2s 10ms/step - loss: 0.6195 - val_loss: 0.6025
Epoch 10/50
250/250 [==============================] - 2s 9ms/step - loss: 0.6140 - val_loss

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [53]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [54]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [24]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        hes skinny
Spanish (true):  el esta delgado
Spanish (pred):  esta estudiando
-
English:        hes strong
Spanish (true):  el es fuerte
Spanish (pred):  esta estudiando
-
English:        hes stupid
Spanish (true):  el es estupido
Spanish (pred):  esta estudiando
-
English:        hes stupid
Spanish (true):  no le llega agua al tanque
Spanish (pred):  esta estudiando
-
English:        hes stupid
Spanish (true):  es un salame
Spanish (pred):  esta estudiando
-
English:        help me out
Spanish (true):  ayudame
Spanish (pred):  segue esto
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  segue esto
-
English:        help me out
Spanish (true):  echeme la mano
Spanish (pred):  segue esto
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  segue esto
-
English:        here i come
Spanish (true):  aqui vengo
Spanish (pred):  aqui esta el perro
-
English:        here i come
Spanish (true):  ya estoy aqui
Spanish (pr

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [25]:
input_sentence = ['I love you']

input_sequence, token_index = text2sequences(max_encoder_seq_length, input_sentence)

input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence[0])
print('translated sentence is: ' + translated_sentence)

source sentence is: I love you
translated sentence is: espera en tom



# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively? 

- In this section, you need to re-train the model we built in secton 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [84]:
# Randomly partition the dataset to training, validation, and test.

rand_indice = numpy.random.permutation(n_train)

train_indices = rand_indice[: int(n_train * 0.9)]
#valid_indices = rand_indice[int(n_train * 0.7) : int(n_train * 0.9)]
test_indices = rand_indice[int(n_train * 0.9):]

train_input_texts = input_texts[train_indices]
train_target_texts = target_texts[train_indices]

#valid_input_texts = input_texts[valid_indices]
#valid_target_texts = target_texts[valid_indices]

test_input_texts =  input_texts[test_indices]
test_target_texts = target_texts[test_indices]

print('The shape of train_input_texts' + str(train_input_texts.shape))
print('The shape of train_target_txets' + str(train_target_texts.shape))
#print('The shape of valid_input_texts' + str(valid_input_texts.shape))
#print('The shape of valid_target_txets' + str(valid_target_texts.shape))
print('The shape of test_input_texts' + str(test_input_texts.shape))
print('The shape of test_target_txets' + str(test_target_texts.shape))

The shape of train_input_texts(18000,)
The shape of train_target_txets(18000,)
The shape of test_input_texts(2000,)
The shape of test_target_txets(2000,)


In [27]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import SmoothingFunction
bleu_scores = []

for input_text, target_text in tqdm(zip(test_input_texts, test_target_texts)):
  input_text = [input_text]
  test_input_sequence, test_token_index = text2sequences(max_encoder_seq_length, input_text)
  test_input_x = onehot_encode(test_input_sequence, max_encoder_seq_length, num_encoder_tokens)
  try:
    translated_sentence = decode_sequence(test_input_x)
  except KeyError as err:
    continue
  
  score = sentence_bleu(target_text, translated_sentence, smoothing_function=SmoothingFunction().method4)
  print(score)
  bleu_scores.append(score)

print('BLEU score for English to Spanish translation:', numpy.mean(bleu_scores))

0it [00:00, ?it/s]

0.32609949139481476
0.3019840131583437
0.30185714811928493
0.30387537471593623
0.3237292787056823
0.3237292787056823
0.3110083741443856
0.30185714811928493
0.31497004588967864
0.3323696442583564
0.30185714811928493
0.2662591197661367
0.30185714811928493
0.28579596974917193
0.27750413301907145
0.31087665410209075
0.3001539509824141
0.31920541394408547
0.2744535278544658
0.3062414723439234
0.31295625634117463
0.310162708656183
0.31920541394408547
0.3121040838154871
0.3037327570745059
0.31087665410209075
0.27948844239600423
0.2991241046361265
0.27372398047645574
0.29774525957349735
0.2925224248656244
0.29774525957349735
0.30853110330739736
0.28153474186969246
0.3062414723439234
0.29578976092515924
0.3078526118350147
0.30387537471593623
0.29206933749538766
0.3037327570745059
0.3190045951414216
0.3121040838154871
0.3180411965717584
0.31433580214471113
0.2765182925762064
0.2991241046361265
0.30387537471593623
0.30583073122743404
0.31920541394408547
0.31705234751187744
0.31433580214471113
0.2

### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [85]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      train_input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       train_target_texts)



print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (18000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (18000, 55)
shape of target_token_index: 29


In [86]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(18000, 18, 28)
(18000, 55, 30)


In [88]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          validation_split=0.2,
          batch_size=64, epochs=50)

model.save('seq2seq_split.h5')

Epoch 1/50
225/225 [==============================] - 5s 13ms/step - loss: 0.8758 - val_loss: 0.7331
Epoch 2/50
225/225 [==============================] - 2s 9ms/step - loss: 0.7626 - val_loss: 0.6664
Epoch 3/50
225/225 [==============================] - 2s 9ms/step - loss: 0.7370 - val_loss: 0.6352
Epoch 4/50
225/225 [==============================] - 2s 9ms/step - loss: 0.7190 - val_loss: 0.6076
Epoch 5/50
225/225 [==============================] - 2s 9ms/step - loss: 0.7028 - val_loss: 0.5868
Epoch 6/50
225/225 [==============================] - 2s 9ms/step - loss: 0.6915 - val_loss: 0.5692
Epoch 7/50
225/225 [==============================] - 2s 9ms/step - loss: 0.6772 - val_loss: 0.5529
Epoch 8/50
225/225 [==============================] - 2s 9ms/step - loss: 0.6690 - val_loss: 0.5393
Epoch 9/50
225/225 [==============================] - 2s 9ms/step - loss: 0.6590 - val_loss: 0.5275
Epoch 10/50
225/225 [==============================] - 2s 9ms/step - loss: 0.6520 - val_loss: 0.518

### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.

In [89]:
from nltk.translate.bleu_score import sentence_bleu
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import SmoothingFunction
bleu_scores = []

for input_text, target_text in tqdm(zip(test_input_texts, test_target_texts)):
  input_text = [input_text]
  test_input_sequence, test_token_index = text2sequences(max_encoder_seq_length, input_text)
  test_input_x = onehot_encode(test_input_sequence, max_encoder_seq_length, num_encoder_tokens)
  try:
    translated_sentence = decode_sequence(test_input_x)
  except KeyError as err:
    continue
  
  score = sentence_bleu(target_text, translated_sentence, smoothing_function=SmoothingFunction().method4)
  print(score)
  bleu_scores.append(score)

print('BLEU score for English to Spanish translation:', numpy.mean(bleu_scores))

0it [00:00, ?it/s]

0.30387537471593623
0.296666753618522
0.30401559514128346
0.3014207975810586
0.30401559514128346
0.3099432039085672
0.30853110330739736
0.3190045951414216
0.30853110330739736
0.30853110330739736
0.3001539509824141
0.32609949139481476
0.30583073122743404
0.28843465160298554
0.2991241046361265
0.3193092175320899
0.3078526118350147
0.29466415380050603
0.3190045951414216
0.30185714811928493
0.279324725262928
0.30387537471593623
0.3213090888918644
0.2879903284693138
0.29774525957349735
0.25346366816882765
0.28648913646093727
0.3037327570745059
0.3121040838154871
0.3237292787056823
0.2997672137673951
0.27558280889605025
0.3277250371116181
0.30185714811928493
0.3078526118350147
0.3237292787056823
0.30583073122743404
0.31433580214471113
0.30401559514128346
0.29774525957349735
0.29578976092515924
0.30387537471593623
0.32502722425194636
0.26824982050388396
0.31497004588967864
0.26432384920938845
0.3213090888918644
0.30583073122743404
0.28843465160298554
0.30401559514128346
0.3237292787056823
0.2